In [1]:
import pandas as pd
import polars as pl
import toml
from pathlib import Path
from sqlalchemy import create_engine

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.set_option('display.float_format', '{:,.1%}'.format)

In [2]:
# Relative path between notebooks and goruped output directories
output_path = Path(summary_config['sc_run_path']) / summary_config["output_folder"]
survey_path = Path(summary_config['sc_run_path']) / summary_config["survey_folder"]
output_dir = output_path / 'RTP/mode_share'

# Write out CSVs with a numeric value prefix to help organize outputs 
incr = 1

In [3]:
def prep_trip_df(trip):
    df_trip = trip.copy()
    df_trip.rename(columns={'mode':'Mode'}, inplace=True)
    df_trip['Trip Type'] = 'Non-Work'
    df_trip.loc[df_trip['dpurp'] == 'Work', 'Trip Type'] = 'Work'

    # add all tripa
    df = df_trip.copy()
    df['Trip Type'] = 'All Trips'
    df_trip = pd.concat([df_trip.sort_values('Trip Type'), df], ignore_index=True)

    return df_trip

In [4]:
trip_county = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_county.csv'))
trip_rg = prep_trip_df(pd.read_csv(output_path / 'agg/dash/mode_share_rg.csv'))

trip_rgc = pd.read_csv(output_path / 'agg/dash/mode_share_rgc.csv')
# add all RGCs
df = trip_rgc[trip_rgc['hh_rgc'] != 'Not in RGC'].groupby(['dpurp','mode']).sum()[['trexpfac']].reset_index()
df['hh_rgc'] = 'All RGCs'
trip_rgc = pd.concat([df, trip_rgc])
trip_rgc = prep_trip_df(trip_rgc)

person = pd.read_csv(output_path / 'agg/dash/person_geog.csv')

### Region

In [5]:
df1 = trip_county.groupby(['Trip Type','Mode'],as_index=False)['trexpfac'].sum()
df2 = trip_county.groupby(['Trip Type'],as_index=False)['trexpfac'].sum()
df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

df = df1.merge(df2, on='Trip Type')
df['mode_share'] = df['trexpfac'] / df['total_trips']

df = df.pivot_table(columns='Trip Type', index='Mode', values='mode_share')

f_name = str(incr)+'_mode_share_region.csv'
df.to_csv(output_dir / f_name); incr+=1

df

Trip Type,All Trips,Non-Work,Work
Mode,,,
Bike,1.8%,1.8%,1.8%
HOV2,20.0%,21.7%,9.0%
HOV3+,12.6%,13.7%,5.5%
SOV,38.7%,35.1%,62.0%
School Bus,1.4%,1.6%,0.0%
TNC,0.5%,0.4%,1.1%
Transit,4.8%,4.7%,5.0%
Walk,20.2%,20.9%,15.6%


### County

In [6]:
def calc_mode_share(df_trip, geog):
    
    # num trips by mode
    df1 = df_trip.groupby([geog, 'Trip Type', 'Mode'],as_index=False)['trexpfac'].sum()
    # num trips
    df2 = df_trip.groupby([geog, 'Trip Type'],as_index=False)['trexpfac'].sum()
    df2.rename(columns={'trexpfac':'total_trips'}, inplace=True)

    df = df1.merge(df2, on=[geog, 'Trip Type'])
    # mode share
    df['mode_share'] = df['trexpfac'] / df['total_trips']

    # mode share by trip type
    df_all = df.loc[df['Trip Type']=="All Trips"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_work = df.loc[df['Trip Type']=="Work"].pivot_table(columns='Mode', index=geog, values='mode_share')
    df_non_work = df.loc[df['Trip Type']=="Non-Work"].pivot_table(columns='Mode', index=geog, values='mode_share')

    return df_all, df_work, df_non_work


df_all, df_work, df_non_work = calc_mode_share(trip_county,'hh_county')

In [7]:
f_name = str(incr)+'_mode_share_county.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,2.0%,18.6%,11.7%,36.1%,1.3%,0.7%,6.8%,22.9%
Kitsap,1.4%,21.0%,12.5%,44.6%,1.2%,0.3%,2.3%,16.8%
Outside Region,NaN,34.1%,2.4%,58.5%,NaN,NaN,NaN,4.9%
Pierce,1.8%,21.8%,14.0%,40.6%,1.5%,0.3%,1.8%,18.2%
Snohomish,1.6%,21.3%,13.8%,42.0%,1.5%,0.3%,3.2%,16.3%


In [8]:
f_name = str(incr)+'_mode_share_county_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,2.0%,20.2%,12.8%,32.5%,1.5%,0.6%,6.9%,23.6%
Kitsap,1.4%,22.7%,13.5%,41.2%,1.4%,0.3%,1.7%,17.8%
Outside Region,NaN,32.5%,2.5%,60.0%,NaN,NaN,NaN,5.0%
Pierce,1.8%,23.5%,15.1%,37.0%,1.7%,0.3%,1.7%,18.9%
Snohomish,1.6%,23.1%,14.9%,38.1%,1.7%,0.3%,3.2%,17.2%


In [9]:
f_name = str(incr)+'_mode_share_county_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_county,,,,,,,,
King,2.0%,8.6%,5.2%,57.4%,0.0%,1.6%,6.2%,19.0%
Kitsap,1.3%,8.9%,5.2%,67.2%,0.0%,0.3%,6.7%,10.3%
Outside Region,NaN,100.0%,NaN,NaN,NaN,NaN,NaN,NaN
Pierce,1.7%,9.6%,5.9%,67.0%,0.0%,0.4%,2.5%,12.8%
Snohomish,1.5%,9.6%,6.0%,68.5%,0.0%,0.4%,3.6%,10.4%


## Regional Growth Center

In [10]:
df_all, df_work, df_non_work = calc_mode_share(trip_rgc, 'hh_rgc')

f_name = str(incr)+'_mode_share_rgc.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.1%,10.5%,6.6%,18.1%,0.8%,1.8%,7.8%,51.3%
Auburn,2.7%,16.2%,10.7%,23.7%,1.3%,0.7%,5.0%,39.7%
Bellevue,3.1%,7.8%,5.9%,10.1%,0.8%,1.0%,9.2%,62.1%
Bothell Canyon Park,2.0%,23.3%,12.3%,39.6%,1.4%,0.5%,3.4%,17.6%
Bremerton,2.7%,10.5%,6.0%,19.3%,0.7%,0.5%,3.3%,56.9%
Burien,2.1%,16.7%,10.5%,34.1%,1.2%,0.6%,6.0%,28.9%
Everett,3.0%,8.6%,5.7%,14.8%,0.8%,0.7%,3.6%,62.7%
Federal Way,2.4%,17.7%,11.9%,27.1%,1.2%,0.5%,7.5%,31.7%
Greater Downtown Kirkland,2.1%,16.8%,8.4%,31.0%,0.7%,0.8%,17.2%,23.1%


In [11]:
f_name = str(incr)+'_mode_share_rgc_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.1%,11.4%,7.2%,16.0%,1.0%,1.4%,8.0%,51.9%
Auburn,2.5%,17.2%,11.5%,20.7%,1.5%,0.6%,4.6%,41.3%
Bellevue,3.1%,8.5%,6.5%,8.7%,1.0%,0.8%,9.9%,61.5%
Bothell Canyon Park,2.0%,24.9%,13.1%,35.8%,1.6%,0.5%,3.5%,18.7%
Bremerton,2.5%,11.2%,6.4%,17.0%,0.9%,0.5%,2.6%,59.0%
Burien,2.0%,17.7%,11.1%,29.3%,1.4%,0.5%,6.3%,31.7%
Everett,3.0%,9.2%,6.3%,12.8%,1.0%,0.6%,3.7%,63.3%
Federal Way,2.3%,18.6%,12.5%,23.2%,1.4%,0.4%,7.5%,34.0%
Greater Downtown Kirkland,2.0%,18.6%,9.3%,28.7%,0.8%,0.6%,15.0%,25.0%


In [12]:
f_name = str(incr)+'_mode_share_rgc_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rgc,,,,,,,,
All RGCs,3.1%,5.6%,3.2%,29.9%,0.0%,3.7%,6.4%,48.2%
Auburn,3.7%,9.0%,5.2%,44.2%,NaN,1.4%,8.1%,28.4%
Bellevue,3.1%,4.2%,2.4%,17.4%,NaN,2.0%,5.6%,65.4%
Bothell Canyon Park,1.9%,10.4%,5.8%,69.0%,NaN,0.6%,2.8%,9.5%
Bremerton,4.1%,6.1%,3.4%,33.8%,NaN,0.6%,7.9%,44.0%
Burien,2.5%,10.3%,6.2%,65.1%,NaN,1.1%,4.1%,10.7%
Everett,3.4%,5.0%,2.9%,25.5%,NaN,1.2%,2.5%,59.5%
Federal Way,3.1%,10.6%,7.0%,56.4%,NaN,0.9%,7.2%,14.8%
Greater Downtown Kirkland,2.6%,7.5%,3.9%,42.4%,NaN,1.6%,28.1%,13.8%


## Regional Geography

In [13]:
df_all, df_work, df_non_work = calc_mode_share(trip_rg, 'hh_rg_proposed')

In [14]:
f_name = str(incr)+'_mode_share_rg.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.2%,22.7%,15.4%,44.9%,1.7%,0.2%,1.7%,12.1%
Core,1.8%,21.0%,13.5%,39.9%,1.5%,0.3%,4.5%,17.4%
HCT,1.5%,22.2%,14.2%,43.4%,1.6%,0.3%,3.9%,13.0%
Metro,2.5%,15.0%,9.0%,29.4%,1.0%,1.1%,7.8%,34.2%
UU,1.2%,26.0%,17.4%,43.3%,1.6%,0.2%,1.2%,9.3%


In [15]:
f_name = str(incr)+'_mode_share_rg_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,1.3%,24.6%,16.7%,40.6%,2.0%,0.2%,1.6%,13.0%
Core,1.8%,22.7%,14.7%,35.9%,1.8%,0.3%,4.4%,18.5%
HCT,1.5%,24.0%,15.4%,39.3%,1.8%,0.2%,3.8%,14.0%
Metro,2.5%,16.4%,9.8%,26.4%,1.2%,0.9%,7.9%,34.9%
UU,1.2%,27.8%,18.6%,39.6%,1.8%,0.2%,1.1%,9.8%


In [16]:
f_name = str(incr)+'_mode_share_rg_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
hh_rg_proposed,,,,,,,,
CitiesTowns,0.8%,10.4%,6.7%,73.8%,0.0%,0.3%,2.1%,5.9%
Core,1.9%,9.9%,6.1%,66.9%,0.0%,0.5%,4.9%,9.8%
HCT,1.4%,10.0%,6.2%,70.5%,0.0%,0.4%,4.9%,6.6%
Metro,2.6%,7.2%,4.2%,46.7%,0.0%,2.3%,6.7%,30.3%
UU,0.8%,10.3%,6.8%,75.1%,NaN,0.2%,2.1%,4.7%


## Equity Geography

In [17]:
equity_geogs = summary_config['equity_geogs']

trip_equity_geog = pd.DataFrame()

for geog in equity_geogs:
    df = prep_trip_df(pd.read_csv(output_path/ ('agg/dash/mode_share_'+geog+'.csv')))
    df['equity_geog'] = df[geog].map({1:geog,
                                      0:"NOT in "+ geog})
    
    trip_equity_geog = pd.concat([trip_equity_geog, df[['dpurp', 'Mode', 'equity_geog', 'trexpfac', 'Trip Type']]])

df_all, df_work, df_non_work = calc_mode_share(trip_equity_geog, 'equity_geog')

In [18]:
f_name = str(incr)+'_mode_share_equity.csv'
df_all.to_csv(output_dir / f_name); incr+=1

df_all

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.7%,20.5%,13.2%,39.7%,1.4%,0.5%,5.1%,17.8%
NOT in hh_elderly_reg,1.9%,19.5%,12.6%,37.2%,1.4%,0.6%,5.1%,21.6%
NOT in hh_english_reg,1.8%,19.9%,12.4%,39.5%,1.3%,0.5%,4.3%,20.2%
NOT in hh_poverty_50,1.8%,20.1%,12.7%,39.1%,1.4%,0.5%,4.7%,19.7%
NOT in hh_poverty_reg,1.7%,20.8%,13.1%,40.7%,1.4%,0.5%,4.7%,17.2%
NOT in hh_racial_50,1.7%,20.5%,12.8%,40.5%,1.3%,0.4%,4.2%,18.4%
NOT in hh_racial_reg,1.6%,20.8%,12.9%,41.6%,1.3%,0.4%,4.1%,17.3%
NOT in hh_youth_reg,2.1%,17.8%,11.0%,34.8%,1.2%,0.8%,6.1%,26.3%
hh_disability_reg,1.9%,19.3%,12.1%,37.6%,1.3%,0.6%,4.4%,22.8%


In [19]:
f_name = str(incr)+'_mode_share_equity_nonwork.csv'
df_non_work.to_csv(output_dir / f_name); incr+=1

df_non_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.7%,22.3%,14.4%,36.0%,1.7%,0.4%,5.0%,18.5%
NOT in hh_elderly_reg,1.9%,21.2%,13.8%,33.5%,1.6%,0.5%,5.1%,22.4%
NOT in hh_english_reg,1.8%,21.6%,13.5%,35.9%,1.5%,0.5%,4.2%,21.0%
NOT in hh_poverty_50,1.8%,21.8%,13.8%,35.5%,1.6%,0.4%,4.6%,20.4%
NOT in hh_poverty_reg,1.7%,22.6%,14.3%,37.0%,1.6%,0.4%,4.7%,17.8%
NOT in hh_racial_50,1.7%,22.3%,13.9%,36.9%,1.5%,0.4%,4.1%,19.2%
NOT in hh_racial_reg,1.6%,22.6%,14.0%,37.9%,1.5%,0.3%,4.0%,18.1%
NOT in hh_youth_reg,2.1%,19.3%,11.9%,31.6%,1.3%,0.6%,6.1%,27.0%
hh_disability_reg,1.9%,20.9%,13.1%,34.2%,1.5%,0.5%,4.4%,23.6%


In [20]:
f_name = str(incr)+'_mode_share_equity_work.csv'
df_work.to_csv(output_dir / f_name); incr+=1

df_work

Mode,Bike,HOV2,HOV3+,SOV,School Bus,TNC,Transit,Walk
equity_geog,,,,,,,,
NOT in hh_disability_reg,1.7%,9.2%,5.6%,63.5%,0.0%,1.0%,5.4%,13.6%
NOT in hh_elderly_reg,2.0%,9.0%,5.5%,60.5%,0.0%,1.2%,5.1%,16.8%
NOT in hh_english_reg,1.7%,8.9%,5.4%,62.2%,0.0%,1.1%,5.1%,15.6%
NOT in hh_poverty_50,1.8%,9.1%,5.5%,62.5%,0.0%,1.1%,5.0%,15.1%
NOT in hh_poverty_reg,1.6%,9.2%,5.6%,64.3%,0.0%,1.0%,5.2%,13.3%
NOT in hh_racial_50,1.6%,9.2%,5.5%,64.4%,0.0%,0.8%,5.0%,13.5%
NOT in hh_racial_reg,1.6%,9.3%,5.6%,66.0%,0.0%,0.6%,5.2%,11.8%
NOT in hh_youth_reg,2.2%,8.2%,4.9%,55.0%,0.0%,1.6%,6.2%,21.9%
hh_disability_reg,1.9%,8.8%,5.3%,60.2%,0.0%,1.2%,4.6%,17.9%
